In [26]:
# !pip install illustris_python
import sys

import numpy as np
import matplotlib.pyplot as plt
import h5py

import scipy
from scipy.stats import binned_statistic_2d
from scipy.ndimage import gaussian_filter
from scipy.interpolate import InterpolatedUnivariateSpline
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u

# from abacusnbody.analysis.tsc import tsc_parallel
import time
import yaml
import glob
# import json
# import pprint 

# sys.path.append('../../illustrisPython/')
import illustris_python as il 
sys.path.append('/pscratch/sd/l/lindajin/SimulationStacker/src/')
# from tools import numba_tsc_3D, hist2d_numba_seq
from utils import fft_smoothed_map, comoving_to_arcmin, arcmin_to_comoving
from halos import select_massive_halos, halo_ind, filter_edge_halo
DATA_DIR = '/pscratch/sd/l/lindajin/DH_profile_kSZ_WL/data/'


In [27]:
r_v_sims = []
snapshot = 74
projection = 'xy'
v_rec = 300
for sim_index in range(0, 1024):
    sim_path = f'/pscratch/sd/l/lindajin/CAMELS/IllustrisTNG/L50n512_SB35/SB35_{sim_index}'
    haloes_cat = il.groupcat.loadHalos(sim_path, snapshot)
    sub_h = haloes_cat['GroupFirstSub'][:]
    central_subhalo_ids = sub_h[np.where(sub_h >= 0)]
    fields = ['SubhaloMass','SubhaloVel','SubhaloVmaxRad', 'SubhaloMassInMaxRad','SubhaloPos','SubhaloGrNr','SubhaloLen']
    subhalos = il.groupcat.loadSubhalos(sim_path, snapshot, fields=fields)
    haloMass   = subhalos['SubhaloMass'][central_subhalo_ids] * 1e10   
    haloVel   = subhalos['SubhaloVel'][central_subhalo_ids] 
    halo_mask, mass_list = select_massive_halos(haloMass, 50e3, 2.4e-3)

    halovel_1D_list = []
    for j, haloID in enumerate(halo_mask):
        # Get halo velocity for the specified projection
        if projection == 'xy':
            halovel_1D_list.append(haloVel[haloID, 2])
        elif projection == 'xz':
            halovel_1D_list.append(haloVel[haloID, 1])
        elif projection == 'yz':
            halovel_1D_list.append(haloVel[haloID, 0])
    
    # Compute correlation across all halos
    halovel_1D_array = np.array(halovel_1D_list)
    v_true_mean = np.mean(halovel_1D_array)
    v_true_rms = np.sqrt(np.mean((halovel_1D_array)**2))
    # print(f"Sim {sim_index}, Number of halos: {len(halovel_1D_array)}, mean velocity: {v_true_mean}, velocity RMS: {v_true_rms}")

    r_v = v_rec / v_true_rms
    # print(f"Sim {sim_index}, r_v: {r_v}")
    r_v_sims.append(r_v)

np.save(DATA_DIR+'r_v_sims.npy', np.array(r_v_sims))


In [9]:
len(halo_mask)

300

In [30]:
np.array(r_v_sims).shape

(1024,)